In [ ]:
# NOTE: For this notebook to work, the neo4j service must be set up.

import os

from interface import PluginInstanceRedisInterface

# Set up redis credentials
redis_host = 'redis'
redis_port = 6379
redis_password = ''

# When your PluginService is running, you can get your channel value from the Logs, or from the query parameter in your open browser.
# Update this value to match that, so that your commands will run against your live workspace.
redis_channel = os.environ.get('REDIS_CHANNEL')

plugin_instance = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
print('Hello')

In [ ]:
# Test connection to Session
comps = plugin_instance.request_complex_list()
print(comps)

In [ ]:
# neo4j setup
from py2neo import Graph, Node, Relationship

neo4j_uri = 'bolt://neo4j:7687/'
username = 'neo4j'
password = 'yourpasswordhere'
graph = Graph(neo4j_uri, auth=(username, password))


In [ ]:
# Create neo4j database
# with open("assets/neo4j_movie_graph_query.txt") as f:
#     query_string = f.read()
# results = graph.run(query_string)
# data = results.data()

In [ ]:
from nanome.api import structure
from nanome.util import Vector3, Color

def create_node_complex(node, atom_position):
    comp = structure.Complex()
    mol = structure.Molecule()
    chain = structure.Chain()
    res = structure.Residue()
    atom = structure.Atom()

    atom.position = atom_position
    atom.labeled = True

    label_text = ''
    if node.labels.__str__() == ':Movie':
        # Movie node
        label_text = node['title']
        atom.atom_color = Color(0, 255, 0)
    else:
        # Person Node
        label_text = node['name']
        atom.atom_color = Color(0, 0, 255)
    atom.label_text = label_text
    res.add_atom(atom)
    chain.add_residue(res)
    mol.add_chain(chain)
    comp.add_molecule(mol)
    comp.name = atom.label_text
    return comp


In [ ]:
import nanome
from nanome.api.shapes import Label, Line, Shape

def draw_relationship_line(struct1, struct2):
    """Draw line connecting two structs.

    :arg struct1: struct
    :arg struct2: struct
    :arg line_settings: Dict describing shape and color of line based on interaction_type
    """
    line = Line()
    for struct, anchor in zip([struct1, struct2], line.anchors):
        anchor.anchor_type = nanome.util.enums.ShapeAnchorType.Atom
        anchor.target = next(struct.atoms).index
    return line
    

In [ ]:
import random

def get_or_create_comp_node(node, comp_nodes, position):
    label_text = ''
    created = False
    if node.labels.__str__() == ':Movie':
        # Movie node
        label_text = node['title']
    else:
        # Person Node
        label_text = node['name']
    if label_text not in [comp.name for comp in comp_nodes]:
        comp_node = create_node_complex(node, position)
        comp_node = plugin_instance.add_to_workspace([comp_node])[0]
        comp_nodes.append(comp_node)
        created = True
    else:
        comp_node = next(iter(comp for comp in comp_nodes if comp.name == label_text))
    return comp_node, created

def render_results(path_results):
#     position = Vector3(15, 15, 15)
    comp_nodes = []

    for record in path_results:
        path = record['p']
        rels = path.relationships
        path_nodes = []
        for rel in rels:
            start_node = rel.start_node
            end_node = rel.end_node
            min_val = 40
            max_val = 80
            position = Vector3(random.randrange(min_val, max_val), random.randrange(min_val, max_val), random.randrange(min_val, max_val))
            start_comp_node, created = get_or_create_comp_node(start_node, comp_nodes, position)
            position = Vector3(random.randrange(min_val, max_val), random.randrange(min_val, max_val), random.randrange(min_val, max_val))
            end_comp_node, created = get_or_create_comp_node(end_node, comp_nodes, position)
        line = draw_relationship_line(start_comp_node, end_comp_node)
        plugin_instance.upload_shapes([line])
    return comp_nodes


In [ ]:
from nanome.api.structure import Workspace

ws = Workspace()
plugin_instance.update_workspace(ws)
result = graph.run("MATCH p=()-[r:ACTED_IN]->() RETURN p LIMIT 25")
node_list = render_results(result)
